In [ ]:
! pip install -U tokenizer

In [ ]:
! pip install tensorflow==1.15

In [ ]:
import tokenizers

In [ ]:
# Create Vocabulary from the corpus
bwpt = tokenizers.BertWordPieceTokenizer(
    vocab_file=None, # We want to build a vocabulary
    add_special_tokens=True,
    unk_token='[UNK]',
    sep_token='[SEP]',
    cls_token='[CLS]',
    clean_text=True,
    handle_chinese_chars=True, # We have some chinese characters in corpus
    strip_accents=True,
    lowercase=True, # Coz we will use BERT Based uncased not cased.
    wordpieces_prefix='##'
)

In [ ]:
#  ! split -l 250000 hi_dedup.txt hindi_
# Spilt the files using Bash command to train faster
# import glob
# file_list = glob.glob("../hindi_*")

In [ ]:
bwpt.train(
    files=["../input/hindi-oscar-corpus/hi_dedup_1000.txt"],
    vocab_size=30000,    # Default can be tuned
    min_frequency=3,     # only after 3 times we recognize chars
    limit_alphabet=1000, # No of alphabets
    special_tokens=['[PAD]', '[UNK]', '[CLS]', '[MASK]', '[SEP]']
)

In [ ]:
bwpt.save("/kaggle/working/", "hindi")

In [ ]:
cd ../input/bertsrc

In [ ]:
! tail -20 /kaggle/working/hindi-vocab.txt

In [ ]:
# Probability of masking a word masked_lm_prob


In [ ]:
!python create_pretraining_data.py \
    --input_file=/kaggle/input/hindi-oscar-corpus/hi_dedup_1000.txt \
    --output_file=/kaggle/working/tf_examples.tfrecord \
    --vocab_file=/kaggle/working/hindi-vocab.txt \
    --do_lower_case=True \
    --max_seq_length=128 \
    --max_predictions_per_seq=20 \
    --masked_lm_prob=0.15 \
    --random_seed=42 \
    --dupe_factor=5

In [ ]:
# To train on TPU you need to keep them on GCP bucket

In [18]:
# !python run_pretraining.py \
#     --input_file=gs://tf-lang-model/*.tfrecord \
#     --output_dir=gs://tf-lang-model/model/ \
#     --do_train=True \
#     --do_eval=True \
#     --bert_config_file=/kaggle/input/bert-base-uncased/config.json \
#     --train_batch_size=32 \
#     --max_seq_length=128 \
#     --max_predictions_per_seq=20 \
#     --num_train_steps=20 \
#     --num_warmup_steps=10 \
#     --learning_rate=2e-5 \
#     --use_tpu=True \
#     --tpu_name=$TPU_NAME

W0512 13:38:42.043282 140669911852416 module_wrapper.py:139] From run_pretraining.py:407: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0512 13:38:42.043545 140669911852416 module_wrapper.py:139] From run_pretraining.py:407: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0512 13:38:42.043859 140669911852416 module_wrapper.py:139] From /kaggle/input/bertsrc/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0512 13:38:42.045854 140669911852416 module_wrapper.py:139] From run_pretraining.py:414: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

2020-05-12 13:38:42.046216: W tensorflow/core/platform/cloud/google_auth_provider.cc:178] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "Not found: Could not locate the credentials file."

In [17]:
! echo $TPU_NAME

grpc://10.0.0.2:8470
